# Build a Tool-Calling Agentic AI Research Assistant with LangChain

This demo will cover building AI Agents with the legacy LangChain `AgentExecutor`. These are fine for getting started, but for working with more advanced agents and having more finer control, LangChain recommends to use LangGraph, which we cover in other courses.

Agents are systems that use an LLM as a reasoning engine to determine which actions to take and what the inputs to those actions should be. The results of those actions can then be fed back into the agent and it determines whether more actions are needed, or whether it is okay to stop.

![](https://i.imgur.com/1uVnBAm.png)



## Install OpenAI, and LangChain dependencies

In [1]:
!pip install langchain==0.3.14
!pip install langchain-openai==0.3.0
!pip install langchain-community==0.3.14

In [2]:
!pip install markitdown

## Enter Open AI API Key

In [1]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

## Enter Tavily Search API Key

Get a free API key from [here](https://tavily.com/#api)

In [2]:
TAVILY_API_KEY = getpass('Enter Tavily Search API Key: ')

## Enter WeatherAPI API Key

Get a free API key from [here](https://www.weatherapi.com/signup.aspx)

In [3]:
WEATHER_API_KEY = getpass('Enter WeatherAPI API Key: ')

## Setup Environment Variables

In [4]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

## Create Tools

Here we create two custom tools which are wrappers on top of the [Tavily API](https://tavily.com/#api) and [WeatherAPI](https://www.weatherapi.com/)

- Web Search tool with information extraction
- Weather tool

![](https://i.imgur.com/TyPAYXE.png)

In [5]:
from langchain_core.tools import tool
from markitdown import MarkItDown
from langchain_community.tools.tavily_search import TavilySearchResults
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, TimeoutError
import requests
import json
from warnings import filterwarnings
filterwarnings('ignore')

tavily_tool = TavilySearchResults(max_results=5,
                                  search_depth='advanced',
                                  include_answer=False,
                                  include_raw_content=True)
# certain websites won't let you crawl them unless you specify a user-agent
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br"
})
md = MarkItDown(requests_session=session)

@tool
def search_web_extract_info(query: str) -> list:
    """Search the web for a query and extracts useful information from the search links."""
    print('Calling web search tool')
    results = tavily_tool.invoke(query)
    docs = []

    def extract_content(url):
        """Helper function to extract content from a URL."""
        extracted_info = md.convert(url)
        text_title = extracted_info.title.strip()
        text_content = extracted_info.text_content.strip()
        return text_title + '\n' + text_content
    # parallelize execution of different urls
    with ThreadPoolExecutor() as executor:
        for result in tqdm(results):
            try:
                future = executor.submit(extract_content, result['url'])
                # Wait for up to 15 seconds for the task to complete
                content = future.result(timeout=15)
                docs.append(content)
            except TimeoutError:
                print(f"Extraction timed out for url: {result['url']}")
            except Exception as e:
                print(f"Error extracting from url: {result['url']} - {e}")

    return docs


@tool
def get_weather(query: str) -> list:
    """Search weatherapi to get the current weather of the queried location."""
    print('Calling weather tool')
    base_url = "http://api.weatherapi.com/v1/current.json"
    complete_url = f"{base_url}?key={WEATHER_API_KEY}&q={query}"

    response = requests.get(complete_url)
    data = response.json()
    if data.get("location"):
        return data
    else:
        return "Weather Data Not Found"

## Test Tool Calling with LLM

In [6]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [search_web_extract_info, get_weather]

chatgpt_with_tools = chatgpt.bind_tools(tools)

In [7]:
prompt = "Get details of Microsoft's earnings call Q4 2024"
response = chatgpt_with_tools.invoke(prompt)
response.tool_calls

[{'name': 'search_web_extract_info',
  'args': {'query': 'Microsoft earnings call Q4 2024 details'},
  'id': 'call_5zhAcU2t2VdxzdMXslBNAqA2',
  'type': 'tool_call'}]

In [8]:
prompt = "how is the weather in Bangalore today"
response = chatgpt_with_tools.invoke(prompt)
response.tool_calls

[{'name': 'get_weather',
  'args': {'query': 'Bangalore'},
  'id': 'call_obFy86zq2VyLbxxZMuMbzGT4',
  'type': 'tool_call'}]

## Build and Test AI Agent

Now that we have defined the tools and the LLM, we can create the agent. We will be using a tool calling agent to bind the tools to the agent with a prompt. We will also add in the capability to store historical conversations as memory

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYS_PROMPT = """Act as a helpful assistant.
                You run in a loop of Thought, Action, PAUSE, Observation.
                At the end of the loop, you output an Answer.
                Use Thought to describe your thoughts about the question you have been asked.
                Use Action to run one of the actions available to you - then return PAUSE.
                Observation will be the result of running those actions.
                Repeat till you get to the answer for the given user query.

                Use the following workflow format:
                  Question: the input task you must solve
                  Thought: you should always think about what to do
                  Action: the action to take which can be any of the following:
                            - break it into smaller steps if needed
                            - see if you can answer the given task with your trained knowledge
                            - call the most relevant tools at your disposal mentioned below in case you need more information
                  Action Input: the input to the action
                  Observation: the result of the action
                  ... (this Thought/Action/Action Input/Observation can repeat N times)
                  Thought: I now know the final answer
                  Final Answer: the final answer to the original input question

                Tools at your disposal to perform tasks as needed:
                  - get_weather: whenever user asks get the weather of a place.
                  - search_web_extract_info: whenever user asks for specific information or if you don't know the answer.
             """

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history", optional=True),
        ("human", "{query}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="Act as a helpful assistant.\n                You run in a loop of Thought, Action, PAUSE, Observation.\n                At the end of the loop, you output an Answer.\n                Use Thought to describe your thoughts about the question you have been asked.\n                Use Action to run one of the actions available to you - then return PAUSE.\n                Observation will be the result of running those actions.\n                Repeat till you get to the answer for the given user query.\n\n                Use the following workflow format:\n                  Question: the input task you must solve\n                  Thought: you should always think about what to do\n                  Action: the action to take which can be any of the following:\n                            - break it into smaller steps if needed\n                            - see if you can

Now, we can initalize the agent with the LLM, the prompt, and the tools.

The agent is responsible for taking in input and deciding what actions to take.

REMEMBER the Agent does not execute those actions - that is done by the AgentExecutor

Note that we are passing in the model `chatgpt`, not `chatgpt_with_tools`.

That is because `create_tool_calling_agent` will call `.bind_tools` for us under the hood.

This should ideally be used with an LLM which supports tool \ function calling

In [10]:
from langchain.agents import create_tool_calling_agent

chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [search_web_extract_info, get_weather]
agent = create_tool_calling_agent(chatgpt, tools, prompt_template)

Finally, we combine the `agent` (the brains) with the `tools` inside the `AgentExecutor` (which will repeatedly call the agent and execute tools).

In [11]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               early_stopping_method='force',
                               max_iterations=10)

In [12]:
query = """Summarize the key points discussed in Nvidia's Q4 2024 earnings call"""
response = chatgpt.invoke(query)
print(response.content)

As of my last update, Nvidia's Q4 2024 earnings call has not occurred, as my data only goes up to October 2023. Therefore, I cannot provide specific details about the call. However, I can guide you on what to typically expect from such earnings calls:

1. **Financial Performance**: Discussion of revenue, net income, and earnings per share compared to previous quarters and analyst expectations.

2. **Segment Performance**: Insights into how different business segments, such as gaming, data center, professional visualization, and automotive, have performed.

3. **Market Trends**: Commentary on market conditions affecting Nvidia's business, such as demand for GPUs, AI advancements, and competition.

4. **Product Updates**: Announcements or updates on new products, technologies, or partnerships.

5. **Guidance**: Forward-looking statements regarding expected financial performance and strategic priorities for the upcoming quarters.

6. **Q&A Session**: Responses to analysts' questions, whic

In [17]:
from IPython.display import display, Markdown

In [18]:
# query = """Summarize the key points discussed in Nvidia's Q4 2024 earnings call"""
# response = agent_executor.invoke({"query": query})


# display(Markdown(response['output']))

In [16]:
query = """Summarize the key points discussed in Intel's Q4 2024 earnings call"""
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

Calling web search tool


 20%|██        | 1/5 [00:02<00:08,  2.18s/it]

Error extracting from url: https://www.gurufocus.com/news/2675853/intel-corp-intc-q4-2024-earnings-call-highlights-surpassing-guidance-amidst-market-challenges - 'NoneType' object has no attribute 'strip'


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
 40%|████      | 2/5 [00:04<00:06,  2.13s/it]

Error extracting from url: https://www.fool.com/earnings/call-transcripts/2025/01/30/intel-intc-q4-2024-earnings-call-transcript/ - 'NoneType' object has no attribute 'strip'


 60%|██████    | 3/5 [00:08<00:05,  2.94s/it]

Error extracting from url: https://download.intel.com/newsroom/2025/c8e6h3a2/intel-q4-2024y-earnings.pdf - File conversion failed after 1 attempts:
 - PdfConverter threw MissingDependencyException with message: PdfConverter recognized the input as a potential .pdf file, but the dependencies needed to read .pdf files have not been installed. To resolve this error, include the optional dependency [pdf] or [all] when installing MarkItDown. For example:

* pip install markitdown[pdf]
* pip install markitdown[all]
* pip install markitdown[pdf, ...]
* etc.



100%|██████████| 5/5 [00:10<00:00,  2.11s/it]


Intel's Q4 2024 earnings call highlighted several key points:

1. **Financial Performance**:
   - Q4 revenue was $14.3 billion, up 7% sequentially.
   - Non-GAAP gross margin was 42.1%, exceeding guidance by 260 basis points.
   - Earnings per share (EPS) for Q4 was $0.13, slightly above the guidance of $0.12.
   - Operating cash flow for Q4 was $3.2 billion, but adjusted free cash flow was negative $1.5 billion.
   - Full-year 2024 revenue was $53.1 billion, a 2.1% decrease year-over-year.
   - Full-year gross margin was 36%, down 760 basis points.
   - Full-year EPS was negative $0.13, with cash from operations at $8.3 billion and adjusted free cash flow at negative $2.2 billion.

2. **Guidance and Outlook**:
   - Q1 2025 revenue guidance is between $11.7 billion and $12.7 billion.
   - Gross margin for Q1 2025 is expected to be around 36%, with EPS guidance at breakeven on a non-GAAP basis.

3. **Business Segments**:
   - Intel Foundry's Q4 revenue was $4.5 billion, with a significant operating loss of $2.3 billion.
   - Mobileye and Altera reported slight revenue increases sequentially.

4. **Strategic Initiatives**:
   - Intel remains a leader in AI PC CPUs, with plans to ship over 100 million systems by the end of 2025.
   - The launch of Panther Lake on Intel 18A is on track for the second half of 2025.
   - Intel Foundry is progressing with Intel 18A, showing competitive offerings and a healthy RFQ pipeline.

5. **Challenges**:
   - Increased competition in the AI PC category and lack of significant participation in the cloud-based AI data center market.
   - Intel Foundry reported a significant operating loss in 2024.
   - Q1 2025 guidance indicates a sequential revenue decline due to macro uncertainty and seasonality.
   - Gross margins are expected to be under pressure throughout 2025.

6. **Q&A Highlights**:
   - Discussions on closing the gap with competitors through Granite Rapids and the niche market potential of Clearwater Forest.
   - Gross margin pressures due to revenue decline and higher costs, with improvements expected later in the year.
   - A more conservative approach for Intel Foundry Services under new leadership, focusing on cautious capital deployment.

Overall, Intel surpassed its guidance amidst market challenges but faces ongoing competition and financial pressures.

In [ ]:
query = """which company's future outlook looks to be better?
        """
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

To provide an accurate assessment of a company's future outlook, I would need to know which specific companies you are interested in comparing. Please provide the names of the companies you would like to evaluate.

In [ ]:
query = """how is the weather in Bangalore today?
           show detailed statistics
        """
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

Calling weather tool


The weather in Bangalore today is as follows:

- **Temperature**: 16.7°C (62.1°F)
- **Condition**: Partly Cloudy
- **Wind**: 10.8 kph (6.7 mph) from the East
- **Pressure**: 1013.0 mb (29.91 in)
- **Precipitation**: 0.0 mm
- **Humidity**: 92%
- **Cloud Cover**: 28%
- **Feels Like**: 16.7°C (62.1°F)
- **Dew Point**: 15.4°C (59.7°F)
- **Visibility**: 10.0 km (6.0 miles)
- **UV Index**: 0.0
- **Wind Gusts**: 19.0 kph (11.8 mph)

The data was last updated at 6:30 AM local time.

In [ ]:
query = """how is the weather in Dubai today?
           show detailed statistics
        """
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

Calling weather tool


The weather in Dubai today is clear. Here are the detailed statistics:

- **Temperature**: 17.6°C (63.7°F)
- **Condition**: Clear
- **Wind**: 12.3 mph (19.8 kph) from the East (93°)
- **Pressure**: 1021.0 mb (30.16 in)
- **Precipitation**: 0.0 mm (0.0 in)
- **Humidity**: 43%
- **Cloud Cover**: 0%
- **Feels Like**: 17.6°C (63.7°F)
- **Dew Point**: 5.0°C (41.1°F)
- **Visibility**: 10.0 km (6.0 miles)
- **UV Index**: 0.0
- **Wind Gusts**: 21.9 mph (35.2 kph)

The current time in Dubai is 5:04 AM.

In [ ]:
query = """which city is hotter?
        """
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

Question: The user is asking which city is hotter, but they haven't specified the cities they are interested in. I need more information to proceed.

Thought: I need to ask the user for the names of the cities they want to compare in terms of temperature.

Final Answer: Could you please specify the names of the cities you would like to compare to determine which one is hotter?

The agent is doing pretty well but unfortunately it doesn't remember conversations. We will use some user-session based memory to store this and dive deeper into this in the next video.

In [ ]:
import os
from typing import Dict, Any
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain
from langchain.agents import AgentExecutor, create_structured_chat_agent

# Define some simple tool functions
def calculator(expression: str) -> float:
    """Evaluate a mathematical expression."""
    try:
        return eval(expression)
    except Exception as e:
        return f"Error: {str(e)}"

def get_weather(location: str) -> str:
    """Get the current weather for a location."""
    # Mock implementation
    return f"The weather in {location} is sunny with a temperature of 72°F."

def search_database(query: str) -> str:
    """Search a database for information."""
    # Mock database
    database = {
        "python": "Python is a high-level programming language.",
        "langchain": "LangChain is a framework for LLM applications.",
        "llm": "LLM stands for Large Language Model."
    }
    query = query.lower()
    return database.get(query, f"No information found for '{query}'.")

# Create the tools
tools = [
    Tool(
        name="Calculator",
        func=calculator,
        description="Useful for performing mathematical calculations"
    ),
    Tool(
        name="WeatherService",
        func=get_weather,
        description="Get current weather for a location"
    ),
    Tool(
        name="Database",
        func=search_database,
        description="Search the database for information"
    )
]

# Create the LLM
llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo"
)

# Create an LLM chain for deciding which tool to use
tool_selection_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a tool selection assistant. 
    Based on the user's question, determine which tool would be most appropriate.
    Respond with only the tool name: "Calculator", "WeatherService", or "Database".
    If none of these tools can help, respond with "None"."""),
    ("human", "{query}")
])

tool_selection_chain = LLMChain(
    llm=llm,
    prompt=tool_selection_prompt,
    output_key="selected_tool"
)

# Create an LLM chain for formatting tool inputs
tool_input_prompt = ChatPromptTemplate.from_messages([
    ("system", """Extract the specific input needed for the tool from the user's query.
    For Calculator: Extract the mathematical expression.
    For WeatherService: Extract the location name.
    For Database: Extract the search query.
    Provide only the extracted information, nothing else."""),
    ("human", "Tool to use: {selected_tool}\nUser query: {query}")
])

tool_input_chain = LLMChain(
    llm=llm,
    prompt=tool_input_prompt,
    output_key="tool_input"
)

# Create an LLM chain for formatting the final response
response_prompt = ChatPromptTemplate.from_messages([
    ("system", "Create a helpful response to the user's query using the tool's output."),
    ("human", "User query: {query}\nTool used: {selected_tool}\nTool output: {tool_output}")
])

response_chain = LLMChain(
    llm=llm,
    prompt=response_prompt,
    output_key="response"
)

# Function to run the full chain
def process_query(query: str) -> str:
    # First, select which tool to use
    tool_selection_result = tool_selection_chain.invoke({"query": query})
    selected_tool = tool_selection_result["selected_tool"].strip()
    print(f"Selected tool: {selected_tool}")
    
    if selected_tool == "None":
        return "I don't have a tool that can help with this query."
    
    # Next, format the input for the selected tool
    tool_input_result = tool_input_chain.invoke({
        "selected_tool": selected_tool,
        "query": query
    })
    tool_input = tool_input_result["tool_input"].strip()
    print(f"Tool input: {tool_input}")
    
    # Find and run the appropriate tool
    tool_output = "Tool not found"
    for tool in tools:
        if tool.name == selected_tool:
            tool_output = tool.func(tool_input)
            break
    print(f"Tool output: {tool_output}")
    
    # Format the final response
    response_result = response_chain.invoke({
        "query": query,
        "selected_tool": selected_tool,
        "tool_output": tool_output
    })
    return response_result["response"]

# Example usage
def run_examples():
    examples = [
        "What is 234 * 78.5?",
        "What's the weather like in San Francisco?",
        "Tell me about LangChain.",
        "What's the capital of France?"
    ]
    
    for example in examples:
        print(f"\n\nQUERY: {example}")
        print("-" * 50)
        response = process_query(example)
        print("FINAL RESPONSE:")
        print(response)

C:\Users\Acer\AppData\Local\Temp\ipykernel_36092\527297010.py:68: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  tool_selection_chain = LLMChain(


In [ ]:
run_examples()



QUERY: What is 234 * 78.5?
--------------------------------------------------
Selected tool: Calculator
Tool input: 234 * 78.5
Tool output: 18369.0
FINAL RESPONSE:
The result of multiplying 234 by 78.5 is 18,369.0.


QUERY: What's the weather like in San Francisco?
--------------------------------------------------
Selected tool: WeatherService
Tool input: San Francisco
Tool output: The weather in San Francisco is sunny with a temperature of 72°F.
FINAL RESPONSE:
The current weather in San Francisco is sunny with a temperature of 72°F. It seems like a beautiful day to enjoy outdoor activities or explore the city. Remember to stay hydrated and wear sunscreen if you plan to be outside for an extended period of time.


QUERY: Tell me about LangChain.
--------------------------------------------------
Selected tool: None
FINAL RESPONSE:
I don't have a tool that can help with this query.


QUERY: What's the capital of France?
--------------------------------------------------
Selected too